In [1]:
from dotenv import load_dotenv
import pandas as pd
import openai
import ast
import json
import os
import re

load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']
client = openai.OpenAI()

In [14]:
input_path = ''
output_path = '../data/gpt4omini'
output_final_csv_path = "gpt4o-mini_preds_dev.csv"

In [ ]:
### DOMAINS: 
# 0-> URW
# 1-> CC
# 2-> Other

# dataframes = []

# # Loop through each fold file, read it, and append to the list
# for i in range(1, 6):  # from fold1 to fold5
#     file_path = os.path.join(input_path, f'fold{i}_predictions.csv')
#     aux = pd.read_csv(file_path)
#     dataframes.append(aux)

# # Concatenate all dataframes into one
# df = pd.concat(dataframes, ignore_index=True)

# #Testing
# # df = df.head(5)

# print(len(df))
# df.head()

726


,article_id,content,domain,labels,label,language,p_domains
0,A9_BG_5871.txt,Зверство! Руснаците започнаха да режат глави н...,NaN,"{'narrative': ['Other'], 'subnarrative': ['Oth...",2,BG,0
1,BG_3245.txt,Подкрепата за Киев от страна на Запада вече не...,URW,"{'narrative': ['Discrediting the West, Diploma...",0,BG,0
2,A9_BG_8210.txt,Украйна – след войната\r\n\r\nДори след края н...,URW,"{'narrative': ['Speculating war outcomes'], 's...",0,BG,2
3,BG_1423.txt,България да се готви: Учен от Швейцария направ...,CC,"{'narrative': ['Amplifying Climate Fears'], 's...",1,BG,1
4,BG_839.txt,Посланикът на САЩ: България вече не е лесна ми...,NaN,"{'narrative': ['Other'], 'subnarrative': ['Oth...",2,BG,0


In [3]:
### DOMAINS: 
# 0-> URW
# 1-> CC
# 2-> Other

file_path = os.path.join('domain_validation_predictions.csv')  
df = pd.read_csv(file_path)

print(len(df))
df.head()

146


,article_id,content,en_content,language,p_domains
0,A9_BG_2569.txt,В Украйна най-ярко се прояви възраждането на н...,NaN,BG,0
1,A9_BG_2592.txt,Геноцидът над децата в Украйна: Зеленски е при...,NaN,BG,0
2,A9_BG_2663.txt,Немски полковник се изплаши от удара на Русия\...,NaN,BG,0
3,A9_BG_2707.txt,The Telegraph: С ресурсите на Украйна Путин би...,NaN,BG,0
4,A9_BG_2756.txt,Киев подготвя провокация в Херсон\n\nНелегални...,NaN,BG,2


In [219]:
# labels = df["labels"].apply(ast.literal_eval).tolist()
# narratives = [list(set(item['subnarrative'])) for item in labels]


In [4]:
def get_climate_prompt(text):
    return f'''In the following text, identify the core narrative that aligns with the author's perspective. 
Classify it based on the options in the list below. If the narrative in the text fall outside the list, answer "Other".

### Options List (Narratives and subnarratives)
Criticism of climate policies
- Climate policies are ineffective
- Climate policies have negative impact on the economy
- Climate policies are only for profit
- Other

Criticism of institutions and authorities
- Criticism of the EU
- Criticism of international entities
- Criticism of national governments
- Criticism of political organizations and figures
- Other

Climate change is beneficial
- CO2 is beneficial
- Temperature increase is beneficial
- Other

Downplaying climate change
- Climate cycles are natural
- Weather suggests the trend is global cooling
- Temperature increase does not have significant impact
- CO2 concentrations are too small to have an impact
- Human activities do not impact climate change
- Ice is not melting
- Sea levels are not rising
- Humans and nature will adapt to the changes
- Other

Questioning the measurements and science
- Methodologies/metrics used are unreliable/faulty
- Data shows no temperature increase
- Greenhouse effect/carbon dioxide do not drive climate change
- Scientific community is unreliable
- Other

Criticism of climate movement
- Climate movement is alarmist
- Climate movement is corrupt
- Ad hominem attacks on key activists
- Other

Controversy about green technologies
- Renewable energy is dangerous
- Renewable energy is unreliable
- Renewable energy is costly
- Nuclear energy is not climate friendly
- Other

Hidden plots by secret schemes of powerful groups
- Blaming global elites
- Climate agenda has hidden motives
- Other

Amplifying Climate Fears
- Earth will be uninhabitable soon
- Amplifying existing fears of global warming
- Doomsday scenarios for humans
- Whatever we do it is already too late
- Other

Green policies are geopolitical instruments
- Climate-related international relations are abusive/exploitative
- Green activities are a form of neo-colonialism
- Other

### Text
             
{text}

Provide only the most relevant narratives that best fit the text's intent. 
If multiple narratives are equally significant, include them all.
Answer **only** with the classifications and always include narratives and subnarratives.'''

In [5]:
def get_war_prompt(text):
    return f'''In the following text, identify the core narrative that aligns with the author's perspective. 
Classify it based on the options in the list below. If the narrative in the text fall outside the list, answer "Other".

### Options List (Narratives and subnarratives)
Blaming the war on others
- Ukraine is the aggressor
- The West are the aggressors
- Other

Discrediting Ukraine
- Rewriting Ukraine’s history
- Discrediting Ukrainian nation and society
- Discrediting Ukrainian military
- Discrediting Ukrainian government and officials and policies
- Ukraine is a puppet of the West
- Ukraine is a hub for criminal activities
- Ukraine is associated with nazism
- Situation in Ukraine is hopeless
- Other

Russia is the Victim
- The West is russophobic
- Russia actions in Ukraine are only self-defence
- UA is anti-RU extremists
- Other

Praise of Russia
- Praise of Russian military might
- Praise of Russian President Vladimir Putin
- Russia is a guarantor of peace and prosperity
- Russia has international support from a number of countries and people
- Russian invasion has strong national support
- Other

Overpraising the West
- NATO will destroy Russia
- The West belongs in the right side of history
- The West has the strongest international support
- Other

Speculating war outcomes
- Russian army is collapsing
- Russian army will lose all the occupied territories
- Ukrainian army is collapsing
- Other

Discrediting the West, Diplomacy
- The EU is divided
- The West is weak
- The West is overreacting
- The West does not care about Ukraine, only about its interests
- Diplomacy does/will not work
- West is tired of Ukraine
- Other

Negative Consequences for the West
- Sanctions imposed by Western countries will backfire
- The conflict will increase the Ukrainian refugee flows to Europe
- Other

Distrust towards Media
- Western media is an instrument of propaganda
- Ukrainian media cannot be trusted
- Other

Amplifying war-related fears
- By continuing the war we risk WWIII
- Russia will also attack other countries
- There is a real possibility that nuclear weapons will be employed
- NATO should/will directly intervene
- Other

### Text
             
{text}

Provide only the **most** relevant narratives that best fit the text's intent. 
If multiple narratives are equally significant, include them all.
Answer **only** with the classifications and always include narratives and subnarratives.'''

### Generate

In [6]:
def get_response(prompt):
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        top_p=0.95,
        max_completion_tokens=200
    )
    return completion.choices[0].message

In [7]:
generations = []

In [9]:
for idx, row in df.iterrows():
    article_id = row['article_id']
    content = row['content']
    #domain = row['domain']
    #labels = row['labels']
    #label = row['label']
    language = row['language']
    p_domains = row['p_domains']
    
    # Check the value of 'p_domains' and call the appropriate prompt function
    if row['p_domains'] == 0:
        response = get_response(get_war_prompt(content))
    elif row['p_domains'] == 1:
        response = get_response(get_climate_prompt(content))
    else:
        response = 'OTHEROTHER'
    
    generated_texts = response if isinstance(response, str) else response.content
    
    result = {
        "article_id": article_id,
        "content": content,
        #"domain": domain,
        #"labels": labels,
        #"label": label,
        "language" : language,
        "p_domains": p_domains,
        "gpt_output": generated_texts
    }
    
    generations.append(result)
    
    # Ensure output path exists and define output file name
    os.makedirs(output_path, exist_ok=True)
    output_name = 'gpt-4o-mini'
    if '/' in output_name:
        output_name = output_name.split('/')[-1]
        
    # Write results to JSON file
    json_output_path = os.path.join(output_path, f"{output_name}_intermediate.json")
    with open(json_output_path, "w", encoding="utf-8") as f:
        json.dump(generations, f, ensure_ascii=False, indent=4)

### Parse Labels

In [10]:
results = pd.DataFrame(generations)

In [58]:
narratives = {
    "Criticism of climate policies": [
        "Climate policies are ineffective",
        "Climate policies have negative impact on the economy",
        "Climate policies are only for profit",
        "Other"
    ],
    "Criticism of institutions and authorities": [
        "Criticism of the EU",
        "Criticism of international entities",
        "Criticism of national governments",
        "Criticism of political organizations and figures",
        "Other"
    ],
    "Climate change is beneficial": [
        "CO2 is beneficial",
        "Temperature increase is beneficial",
        "Other"
    ],
    "Downplaying climate change": [
        "Climate cycles are natural",
        "Weather suggests the trend is global cooling",
        "Temperature increase does not have significant impact",
        "CO2 concentrations are too small to have an impact",
        "Human activities do not impact climate change",
        "Ice is not melting",
        "Sea levels are not rising",
        "Humans and nature will adapt to the changes",
        "Other"
    ],
    "Questioning the measurements and science": [
        "Methodologies/metrics used are unreliable/faulty",
        "Data shows no temperature increase",
        "Greenhouse effect/carbon dioxide do not drive climate change",
        "Scientific community is unreliable",
        "Other"
    ],
    "Criticism of climate movement": [
        "Climate movement is alarmist",
        "Climate movement is corrupt",
        "Ad hominem attacks on key activists",
        "Other"
    ],
    "Controversy about green technologies": [
        "Renewable energy is dangerous",
        "Renewable energy is unreliable",
        "Renewable energy is costly",
        "Nuclear energy is not climate friendly",
        "Other"
    ],
    "Hidden plots by secret schemes of powerful groups": [
        "Blaming global elites",
        "Climate agenda has hidden motives",
        "Other"
    ],
    "Amplifying Climate Fears": [
        "Earth will be uninhabitable soon",
        "Amplifying existing fears of global warming",
        "Doomsday scenarios for humans",
        "Whatever we do it is already too late",
        "Other"
    ],
    "Green policies are geopolitical instruments": [
        "Climate-related international relations are abusive/exploitative",
        "Green activities are a form of neo-colonialism",
        "Other"
    ],
    "Blaming the war on others": [
        "Ukraine is the aggressor",
        "The West are the aggressors",
        "Other"
    ],
    "Discrediting Ukraine": [
        "Rewriting Ukraine’s history",
        "Discrediting Ukrainian nation and society",
        "Discrediting Ukrainian military",
        "Discrediting Ukrainian government and officials and policies",
        "Ukraine is a puppet of the West",
        "Ukraine is a hub for criminal activities",
        "Ukraine is associated with nazism",
        "Situation in Ukraine is hopeless",
        "Other"
    ],
    "Russia is the Victim": [
        "The West is russophobic",
        "Russia actions in Ukraine are only self-defence",
        "UA is anti-RU extremists",
        "Other"
    ],
    "Praise of Russia": [
        "Praise of Russian military might",
        "Praise of Russian President Vladimir Putin",
        "Russia is a guarantor of peace and prosperity",
        "Russia has international support from a number of countries and people",
        "Russian invasion has strong national support",
        "Other"
    ],
    "Overpraising the West": [
        "NATO will destroy Russia",
        "The West belongs in the right side of history",
        "The West has the strongest international support",
        "Other"
    ],
    "Speculating war outcomes": [
        "Russian army is collapsing",
        "Russian army will lose all the occupied territories",
        "Ukrainian army is collapsing",
        "Other"
    ],
    "Discrediting the West, Diplomacy": [
        "The EU is divided",
        "The West is weak",
        "The West is overreacting",
        "The West does not care about Ukraine, only about its interests",
        "Diplomacy does/will not work",
        "West is tired of Ukraine",
        "Other"
    ],
    "Negative Consequences for the West": [
        "Sanctions imposed by Western countries will backfire",
        "The conflict will increase the Ukrainian refugee flows to Europe",
        "Other"
    ],
    "Distrust towards Media": [
        "Western media is an instrument of propaganda",
        "Ukrainian media cannot be trusted",
        "Other"
    ],
    "Amplifying war-related fears": [
        "By continuing the war we risk WWIII",
        "Russia will also attack other countries",
        "There is a real possibility that nuclear weapons will be employed",
        "NATO should/will directly intervene",
        "Other"
    ]
}

# Function to detect narratives and subnarratives in text
def detect_narratives(text):
    detected_narrative = set()       # Use sets to avoid duplicates
    detected_subnarrative = set()
    
    # Iterate through each narrative and its subnarratives
    for narrative, subnarratives in narratives.items():
        # Flag to check if the narrative is detected in the text
        narrative_detected = False
        # Flag to check if any subnarrative is detected
        subnarrative_detected = False
        
        # Check for mentions of the narrative
        if re.search(r'\b' + re.escape(narrative) + r'\b', text, re.IGNORECASE):
            narrative_detected = True
            detected_narrative.add(narrative)
        
            # Check if the narrative has any subnarratives associated
            if subnarratives:
                # Check for mentions of each subnarrative
                for subnarrative in subnarratives:
                    if re.search(r'\b' + re.escape(subnarrative) + r'\b', text, re.IGNORECASE):
                        detected_subnarrative.add(subnarrative)
                        subnarrative_detected = True  # A subnarrative was detected
                
                # Special case: Narrative has subnarratives but none were detected
                if not subnarrative_detected:
                    # Check if 'Other' is NOT among the subnarratives
                    if 'Other' not in subnarratives:
                        adjusted_subnarrative = f"{narrative}: Other"
                        detected_subnarrative.add(adjusted_subnarrative)
            else:
                # Special case: Narrative has no subnarratives associated
                adjusted_subnarrative = f"{narrative}: Other"
                detected_subnarrative.add(adjusted_subnarrative)
    
    # Default to "Other" if no narrative or subnarrative is detected
    if not detected_narrative:
        detected_narrative.add("Other")
    if not detected_subnarrative:
        detected_subnarrative.add("Other")
        
    # Convert sets back to lists for the final output
    return {
        "narrative": list(detected_narrative),
        "subnarrative": list(detected_subnarrative)
    }

In [59]:
p_system = [
    detect_narratives(row['gpt_output']) if row['p_domains'] != 2 else {'narrative': ['Other'], 'subnarrative': ['Other']}
    for _, row in results.iterrows()
]

print(sum(1 for item in p_system if item == {'narrative': ['Other'], 'subnarrative': ['Other']}))
print(sum(1 for item in p_system if item != {'narrative': ['Other'], 'subnarrative': ['Other']}))
print(sum(1 for item in p_system if item == {'narrative': ['Other'], 'subnarrative': ['Other']}) + sum(1 for item in p_system if item != {'narrative': ['Other'], 'subnarrative': ['Other']}))

33
113
146


In [60]:
results['p_system'] = p_system

# Define the output path for the new CSV file


# Save the DataFrame as a CSV file
results.to_csv(output_final_csv_path, index=False, encoding='utf-8')

print(f"CSV file created at: {output_final_csv_path}")

CSV file created at: gpt4o-mini_preds_dev.csv


### Metrics

In [229]:
results

,article_id,content,domain,labels,label,language,p_domains,gpt_output,p_system
0,A9_BG_5871.txt,Зверство! Руснаците започнаха да режат глави н...,NaN,"{'narrative': ['Other'], 'subnarrative': ['Oth...",2,BG,0,Discrediting Russia \n- Discrediting Russian ...,"{'narrative': ['Other'], 'subnarrative': ['Oth..."
1,BG_3245.txt,Подкрепата за Киев от страна на Запада вече не...,URW,"{'narrative': ['Discrediting the West, Diploma...",0,BG,0,"- Discrediting the West, Diplomacy\n - The We...","{'narrative': ['Discrediting the West, Diploma..."
2,A9_BG_8210.txt,Украйна – след войната\r\n\r\nДори след края н...,URW,"{'narrative': ['Speculating war outcomes'], 's...",0,BG,2,OTHEROTHER,"{'narrative': ['Other'], 'subnarrative': ['Oth..."
3,BG_1423.txt,България да се готви: Учен от Швейцария направ...,CC,"{'narrative': ['Amplifying Climate Fears'], 's...",1,BG,1,Amplifying Climate Fears\n- Earth will be unin...,"{'narrative': ['Downplaying climate change', '..."
4,BG_839.txt,Посланикът на САЩ: България вече не е лесна ми...,NaN,"{'narrative': ['Other'], 'subnarrative': ['Oth...",2,BG,0,"- Discrediting the West, Diplomacy\n - The We...","{'narrative': ['Discrediting the West, Diploma..."
...,...,...,...,...,...,...,...,...,...
721,PT_150.txt,"Calor extremo, incêndios e chuva que não cai: ...",CC,"{'narrative': ['Amplifying Climate Fears', 'Am...",1,PT,1,- Criticism of climate policies\n - Climate p...,{'narrative': ['Green policies are geopolitica...
722,PT_116.txt,Amorim: Assassinato de líder do Hamas foi 'exc...,URW,"{'narrative': ['Discrediting Ukraine'], 'subna...",0,PT,0,"- Discrediting the West, Diplomacy\n - The We...","{'narrative': ['Discrediting the West, Diploma..."
723,PT_159.txt,A transição energética\r\n\r\nMultiplicam-se o...,CC,{'narrative': ['Criticism of institutions and ...,1,PT,1,- Amplifying Climate Fears\n - Earth will be ...,{'narrative': ['Criticism of climate policies'...
724,PT_74.txt,Rússia assume controlo de mais uma povoação no...,URW,"{'narrative': ['Praise of Russia'], 'subnarrat...",0,PT,0,- Speculating war outcomes\n- Discrediting Ukr...,"{'narrative': ['Discrediting the West, Diploma..."


In [240]:
true_labels = results["labels"].apply(ast.literal_eval).tolist()  # Still needed for labels if they are strings
true_narratives = [set(item['narrative']) for item in true_labels]
true_subnarratives = [set(item['subnarrative']) for item in true_labels]

# Extract p_system directly as lists of sets for narratives and subnarratives (no literal_eval needed)
pred_narratives = [set(item['narrative']) for item in results["p_system"]]
pred_subnarratives = [set(item['subnarrative']) for item in results["p_system"]]

In [242]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import MultiLabelBinarizer

# Step 1: Extract all unique narratives and subnarratives
all_narratives = set().union(*true_narratives, *pred_narratives)
all_subnarratives = set().union(*true_subnarratives, *pred_subnarratives)

# Step 2: Initialize MultiLabelBinarizer for both narratives and subnarratives
narrative_binarizer = MultiLabelBinarizer(classes=list(all_narratives))
subnarrative_binarizer = MultiLabelBinarizer(classes=list(all_subnarratives))

# Step 3: Transform true and predicted labels into binary indicator format
true_narratives_binary = narrative_binarizer.fit_transform(true_narratives)
pred_narratives_binary = narrative_binarizer.transform(pred_narratives)
true_subnarratives_binary = subnarrative_binarizer.fit_transform(true_subnarratives)
pred_subnarratives_binary = subnarrative_binarizer.transform(pred_subnarratives)

# Step 4: Calculate metrics using scikit-learn
# Narratives metrics
narrative_f1 = f1_score(true_narratives_binary, pred_narratives_binary, average='samples')
narrative_precision = precision_score(true_narratives_binary, pred_narratives_binary, average='samples')
narrative_recall = recall_score(true_narratives_binary, pred_narratives_binary, average='samples')

# Subnarratives metrics
subnarrative_f1 = f1_score(true_subnarratives_binary, pred_subnarratives_binary, average='samples')
subnarrative_precision = precision_score(true_subnarratives_binary, pred_subnarratives_binary, average='samples')
subnarrative_recall = recall_score(true_subnarratives_binary, pred_subnarratives_binary, average='samples')

# Display results
print("Narratives - F1 (samples):", narrative_f1)
print("Narratives - Precision (samples):", narrative_precision)
print("Narratives - Recall (samples):", narrative_recall)
print()
print("Subnarratives - F1 (samples):", subnarrative_f1)
print("Subnarratives - Precision (samples):", subnarrative_precision)
print("Subnarratives - Recall (samples):", subnarrative_recall)



Narratives - F1 (samples): 0.4852959941802917
Narratives - Precision (samples): 0.4882460973370064
Narratives - Recall (samples): 0.5455758887577069

Subnarratives - F1 (samples): 0.443255743821207
Subnarratives - Precision (samples): 0.4424717958808868
Subnarratives - Recall (samples): 0.5162250207704753
